### Example uses of SPARQL querying with the Environment Agency Real-Time Flood Management APIs
The OpenLink Virtuoso SPARQL processor is used for querying RDF data provided by the Environment Agency and for federated queries with other SPARQL endpoints.
##### Dependencies
Python3 (Requires Jupyter, SPARQLWrapper, Folium and Ipyleaflet packages)<br>
[OpenLink Virtuoso SPARQL Processor](https://virtuoso.openlinksw.com)

### Example 1

Investigate available properties of the Environment Agency datasets at:

[Flood Monitoring Stations](http://environment.data.gov.uk/flood-monitoring/id/stations.ttl)

[Flood Monitoring Measures](http://environment.data.gov.uk/flood-monitoring/id/measures.ttl)

[Flood Monitoring Readings](http://environment.data.gov.uk/flood-monitoring/data/readings.ttl)


In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON

sparql_query = """
PREFIX rt:<http://environment.data.gov.uk/flood-monitoring/def/core/>
PREFIX rdfs:<http://www.w3.org/2000/01/rdf-schema#>
PREFIX geo:<http://www.w3.org/2003/01/geo/wgs84_pos#>

SELECT DISTINCT ?property 
FROM <http://environment.data.gov.uk/flood-monitoring/id/stations.rdf>
#FROM <http://environment.data.gov.uk/flood-monitoring/id/measures.rdf>
#FROM <http://environment.data.gov.uk/flood-monitoring/data/readings.rdf>
WHERE {
  ?s ?property ?o
}
"""
# Assumes OpenLink Virtuoso sparql endpoint running locally at:
sparql_endpoint = "http://localhost:8890/sparql"

sparql = SPARQLWrapper(sparql_endpoint)
sparql.setQuery(sparql_query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
for result in results["results"]["bindings"]:
    print (result["property"]["value"])


### Example 2
Discover all catchment areas in the Environment Agency datasets using the *rt:catchmentName* property.

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON

sparql_query = """
PREFIX rt:<http://environment.data.gov.uk/flood-monitoring/def/core/>

SELECT DISTINCT ?catchmentName
FROM <http://environment.data.gov.uk/flood-monitoring/id/stations.rdf>
WHERE {
  ?s rt:catchmentName ?catchmentName .
}
"""

# Assumes OpenLink Virtuoso sparql endpoint running locally at:
sparql_endpoint = "http://localhost:8890/sparql"

sparql = SPARQLWrapper(sparql_endpoint)
sparql.setQuery(sparql_query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
for result in results["results"]["bindings"]:
    print (result["catchmentName"]["value"])


### Example 3
Discover all monitoring stations in "Lune and Wyre" catchment area and display *rt:town, rdfs:label, geo:long, geo:lat, rt:latestReading* properties.

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON

sparql_query = """
PREFIX rt:<http://environment.data.gov.uk/flood-monitoring/def/core/>
PREFIX rdfs:<http://www.w3.org/2000/01/rdf-schema#>
PREFIX geo:<http://www.w3.org/2003/01/geo/wgs84_pos#>

SELECT ?label ?town ?lon ?lat ?s ?measures ?latestReading ?latestValue
FROM <http://environment.data.gov.uk/flood-monitoring/id/stations.rdf>
FROM <http://environment.data.gov.uk/flood-monitoring/id/measures.rdf>
FROM <http://environment.data.gov.uk/flood-monitoring/data/readings.rdf>
WHERE {
  ?s rt:town ?town ;
     geo:long ?lon ;
     geo:lat ?lat ;
     rt:catchmentName "Lune and Wyre" ; 
     rdfs:label ?label ;
     rt:measures ?measures . 
     ?measures rt:latestReading ?latestReading . 
     ?latestReading rt:value ?latestValue .
}
"""

# Assumes OpenLink Virtuoso sparql endpoint running locally at:
sparql_endpoint = "http://localhost:8890/sparql"

sparql = SPARQLWrapper(sparql_endpoint)
sparql.setQuery(sparql_query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
for result in results["results"]["bindings"]:
    print("{}, {}, {} {}, {}".format(result["label"]["value"],
                                 result["town"]["value"],
                                 result["lon"]["value"],
                                 result["lat"]["value"],
                                 result["latestValue"]["value"]))


### Example 4
Discover all monitoring stations in "Eden and Esk" catchment area and display *rt:town, rdfs:label, geo:long, geo:lat, rt:latestReading* properties.

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON

sparql_query = """
PREFIX rt:<http://environment.data.gov.uk/flood-monitoring/def/core/>
PREFIX rdfs:<http://www.w3.org/2000/01/rdf-schema#>
PREFIX geo:<http://www.w3.org/2003/01/geo/wgs84_pos#>

SELECT ?label ?town ?lon ?lat ?s ?measures ?latestReading ?latestValue
FROM <http://environment.data.gov.uk/flood-monitoring/id/stations.rdf>
FROM <http://environment.data.gov.uk/flood-monitoring/id/measures.rdf>
FROM <http://environment.data.gov.uk/flood-monitoring/data/readings.rdf>
WHERE {
  ?s rt:town ?town ;
     geo:long ?lon ;
     geo:lat ?lat ;
     rt:catchmentName "Eden and Esk" ; 
     rdfs:label ?label ;
     rt:measures ?measures . 
     ?measures rt:latestReading ?latestReading . 
     ?latestReading rt:value ?latestValue .
}
"""

# Assumes OpenLink Virtuoso sparql endpoint running locally at:
sparql_endpoint = "http://localhost:8890/sparql"

sparql = SPARQLWrapper(sparql_endpoint)
sparql.setQuery(sparql_query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

for result in results["results"]["bindings"]:
    print("{}, {}, {} {}, {}".format(result["label"]["value"],
                                 result["town"]["value"],
                                 result["lon"]["value"],
                                 result["lat"]["value"],
                                 result["latestValue"]["value"]))



### Example 5
Discover all stations in "Lune and Wyre" catchment and plot locations on an OSM map using the Folium python package.

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON
import folium

sparql_query = """
PREFIX rt:<http://environment.data.gov.uk/flood-monitoring/def/core/>
PREFIX rdfs:<http://www.w3.org/2000/01/rdf-schema#>
PREFIX geo:<http://www.w3.org/2003/01/geo/wgs84_pos#>

SELECT ?label ?town ?lon ?lat ?s ?measures ?latestReading ?latestValue
FROM <http://environment.data.gov.uk/flood-monitoring/id/stations.rdf>
FROM <http://environment.data.gov.uk/flood-monitoring/id/measures.rdf>
FROM <http://environment.data.gov.uk/flood-monitoring/data/readings.rdf>
WHERE {
  ?s rt:town ?town ;
     geo:long ?lon ;
     geo:lat ?lat ;
     rt:catchmentName "Lune and Wyre" ; 
     rdfs:label ?label ;
     rt:measures ?measures . 
     ?measures rt:latestReading ?latestReading . 
     ?latestReading rt:value ?latestValue .
}
"""

# Assumes OpenLink Virtuoso sparql endpoint running locally at:
sparql_endpoint = "http://localhost:8890/sparql"

sparql = SPARQLWrapper(sparql_endpoint)
sparql.setQuery(sparql_query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

map_osm = folium.Map(location=[54.045675,-2.8009287])
for result in results["results"]["bindings"]:
    folium.Marker(location=[result["lat"]["value"],
                  result["lon"]["value"]], 
                  popup=result["label"]["value"]).add_to(map_osm)
map_osm


### Example 6
Draw boundary polygon on map to discover all monitoring stations within the boundary shape using the Environment Agency API endpoints. Uses the Ipyleaflet python package.

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON
from ipyleaflet import (
    Map,
    Marker,
    TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle, Circle, CircleMarker,
    GeoJSON,
    DrawControl
)
from traitlets import link

def wkt_convert(c):
    return str(c[0]) + " " + str(c[1])

def handle_draw(self, action, geo_json):
    global leaflet_map
    coords = geo_json['geometry']['coordinates'][0]
    wkt = "POLYGON((%s " % ', '.join(map(wkt_convert, coords)) + "))"
    sparql_query = """
            PREFIX rt:<http://environment.data.gov.uk/flood-monitoring/def/core/>
            PREFIX rdfs:<http://www.w3.org/2000/01/rdf-schema#>
            PREFIX geo:<http://www.w3.org/2003/01/geo/wgs84_pos#>

            SELECT ?label ?town ?lon ?lat ?s 
            FROM <http://environment.data.gov.uk/flood-monitoring/id/stations.rdf>
            FROM <http://environment.data.gov.uk/flood-monitoring/id/measures.rdf>
            FROM <http://environment.data.gov.uk/flood-monitoring/data/readings.rdf>
            WHERE {{
                ?s rt:town ?town ;
                   geo:long ?lon ;
                   geo:lat ?lat ; 
                   rdfs:label ?label .
                 FILTER( bif:st_intersects( bif:st_point(?lon, ?lat), bif:st_geomfromtext({}),0.0)) .
            }}
    """.format("\"{}\"".format(wkt))
    
    # Assumes OpenLink Virtuoso sparql endpoint running locally at:
    sparql_endpoint = "http://localhost:8890/sparql"

    sparql = SPARQLWrapper(sparql_endpoint)
    sparql.setQuery(sparql_query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    for result in results["results"]["bindings"]:
        mark = Marker(location=[result['lat']['value'], result['lon']['value']], title=result['town']['value'])
        leaflet_map.add_layer(mark)

center = [54.047186,-2.801706]
zoom = 10
leaflet_map = Map(default_tiles=TileLayer(opacity=1.0), center=center, zoom=zoom)
dc = DrawControl()
dc.on_draw(handle_draw)
leaflet_map.add_control(dc)
leaflet_map

### Example 7
Demonstrates a federated query making use of a remote SPARQL endpoint (*http://dbpedia.org/sparql*). 
Discover all dbpedia places within 0.5km of geospatial point and plot on an osm map.

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON
import folium

sparql_query = """
SELECT DISTINCT ?resource ?label ?long ?lat
WHERE 
{ 
    SERVICE <http://dbpedia.org/sparql>
    {
        ?resource geo:geometry ?location ;
                  rdfs:label ?label . 
        FILTER( bif:st_intersects( ?location, bif:st_point(-2.8009287, 54.045675), 0.5)) .
        BIND(bif:st_x(?location) AS ?long) .
        BIND(bif:st_y(?location) AS ?lat) .
    }
}
"""

# Assumes OpenLink Virtuoso sparql endpoint running locally at:
sparql_endpoint = "http://localhost:8890/sparql"

sparql = SPARQLWrapper(sparql_endpoint)
sparql.setQuery(sparql_query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

map_osm = folium.Map(location=[54.045675,-2.8009287])
for result in results["results"]["bindings"]:
    folium.Marker(location=[result["lat"]["value"],
                  result["long"]["value"]], 
                  popup=result["label"]["value"]).add_to(map_osm)
map_osm


### Example 8
Demonstrates a federated query making use of a remote SPARQL endpoint. 
Discover all dbpedia places within a drawn polygon bounding area and plot on a map using the Ipyleaflet package.

In [ ]:

from SPARQLWrapper import SPARQLWrapper, JSON
from ipyleaflet import (
    Map,
    Marker,
    TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle, Circle, CircleMarker,
    GeoJSON,
    DrawControl
)
from traitlets import link

def wkt_convert(c):
    return str(c[0]) + " " + str(c[1])

def handle_draw(self, action, geo_json):
    global leaflet_map
    coords = geo_json['geometry']['coordinates'][0]
    wkt = "POLYGON((%s " % ', '.join(map(wkt_convert, coords)) + "))"
    sparql_query = """
                    SELECT DISTINCT ?resource ?label ?long ?lat
                    WHERE 
                    {{ 
                        SERVICE <http://dbpedia.org/sparql>
                        {{
                                ?resource geo:geometry ?location ;
                                          rdfs:label ?label . 
                                FILTER( bif:st_intersects( ?location, bif:st_geomfromtext({}),0.0)) .
                                FILTER (lang(?label) = 'en') .
                                BIND(bif:st_x(?location) AS ?long) .
                                BIND(bif:st_y(?location) AS ?lat) .
                        }}
                    }}
                    """.format("\"{}\"".format(wkt))
    
    # Assumes OpenLink Virtuoso sparql endpoint running locally at:
    sparql_endpoint = "http://localhost:8890/sparql"

    sparql = SPARQLWrapper(sparql_endpoint)
    sparql.setQuery(sparql_query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    for result in results["results"]["bindings"]:
        mark = Marker(location=[result['lat']['value'], result['long']['value']], title=result['label']['value'])
        leaflet_map.add_layer(mark)

center = [54.047186,-2.801706]
zoom = 15
leaflet_map = Map(default_tiles=TileLayer(opacity=1.0), center=center, zoom=zoom)
dc = DrawControl()
dc.on_draw(handle_draw)
leaflet_map.add_control(dc)
leaflet_map

### Example 9

Embed a separate web application in a notebook using an iframe.

In [ ]:
import IPython
url = 'http://scc-digsci.lancs.ac.uk'
iframe = '<iframe src=' + url + ' width=1000 height=1000></iframe>'
IPython.display.HTML(iframe)